In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import math
from sympy import symbols, Eq, solve
from scipy.optimize import fsolve


In [4]:
import ipywidgets as widgets
from IPython.display import display
uploader = widgets.FileUpload()
display(uploader)

FileUpload(value=(), description='Upload')

In [ ]:
warranty_DC = pd.

In [2]:
def failure_model_w_CI(sku):
    data = warranty_DC.copy()
    newdata = pd.DataFrame()
    if sku not in warranty_DC['parent_Parent_Sku']:
        newdata = warranty_DC[warranty_DC['asset_Finish__c'] == sku]
    else:
        newdata = warranty_DC[warranty_DC['parent_Parent_Sku'] == sku]

    sortedData = newdata['days_diff'].dropna().sort_values(ascending = True)
    sortedData = pd.DataFrame(sortedData)
    sortedData = sortedData[sortedData['days_diff'] <= 2160]
    timeSum = float(sortedData.sum().iloc[0])
    beta = symbols("beta")
    n = len(sortedData)
    bVal = sortedData.iloc[-1]


    #The following two lines of code aren't technically needed, but they fix "errors" that occur
    bVal = float(bVal.iloc[0]) if isinstance(bVal, (np.ndarray, pd.Series)) else bVal
    beta = float(beta.iloc[0]) if isinstance(beta, (np.ndarray, pd.Series)) else beta

    
    def equation_beta(beta):
        return timeSum + (n/beta) - (n * bVal)/(1 - math.exp(-beta * bVal))

    x1 = fsolve(equation_beta, 0.001)
    beta = x1[0]
    alpha = math.log((n*(beta))/((math.e)**(beta*bVal)-1))

    
    def f(t):
        return (np.exp(alpha + beta * t) - np.exp(alpha)) / beta 

    def CI(t, beta, alpha):
        return (np.exp(alpha + beta * t) - np.exp(alpha)) / beta 

    
    failure_counts = sortedData['days_diff'].value_counts().sort_index()
    cumulative_failures = failure_counts.cumsum()
    
    counts = sortedData['days_diff'].value_counts().sort_index()
    pdf = counts / counts.sum()

    x = np.linspace(0, 2160, 2160)
    y = f(x)
    x1 = cumulative_failures.index  # Months
    y1 = cumulative_failures.values  
    

    #STUFF I TOOK FROM SAKIB
    fisher_info = n / (beta**2) - (n * bVal**2 * np.exp(-beta * bVal)) / ((1 - np.exp(-beta * bVal))**2)
    se_beta = np.sqrt(1 / fisher_info)
    z_score = 1.96
    beta_hat_ci_lower = beta - z_score * se_beta
    beta_hat_ci_upper = beta + z_score * se_beta
    alpha_hat_ci_lower = np.log((n * beta_hat_ci_lower) / (np.exp(beta_hat_ci_lower * bVal) - 1))
    alpha_hat_ci_upper = np.log((n * beta_hat_ci_upper) / (np.exp(beta_hat_ci_upper * bVal) - 1))

    #added below code because without it I was getting errors and this fixes it
    #don't ask me why
    beta_hat_ci_lower = float(beta_hat_ci_lower)
    beta_hat_ci_upper = float(beta_hat_ci_upper)
    alpha_hat_ci_lower = float(alpha_hat_ci_lower)
    alpha_hat_ci_upper = float(alpha_hat_ci_upper)
    
    

  

    y_predicted_ci_lower = CI(np.array(x), beta_hat_ci_lower, alpha_hat_ci_lower)
    y_predicted_ci_upper = CI(np.array(x), beta_hat_ci_upper, alpha_hat_ci_upper)

    plt.figure(figsize=(12, 7)) 
    
    plt.fill_between(
        x,
        y_predicted_ci_lower,
        y_predicted_ci_upper,
        color="#e31a1c",
        alpha=0.3,
        label="95% CI",
    )
    ##################################################


    # Plot the first curve (predicted function)
    plt.plot(x, y, color='red', linestyle='-', label="Predicted Curve")
    
    # Plot the second curve (cumulative failures)
    plt.plot(x1, y1, color='blue', linestyle='-', label="Actual Cumulative Failures")
    
    # Labels and title
    plt.xlabel("Time (Days)")
    plt.ylabel("Cumulative Failures")
    plt.title(f"Predicted vs. Actual Cumulative Failures for {sku}")
    
    # Add grid and legend
    plt.grid(True)
    plt.legend()
    
    # Show the combined plot
    plt.show()



    #ADDED 3/13/2025
    predicted_y_at_x1 = np.interp(x1, x, y)
    #basically makes it so that we have the predicted values at the exact days, so we can compare it w/ the actual y values, no extra values exist.

    
    # Create dataframe for actual data
    df_combined = pd.DataFrame({
    "sku": sku,
    "source": "Actual",
    "time": x1,
    "actual_cumulative_failures": y1,
    "predicted_cumulative_failures": predicted_y_at_x1
    })
    
    return df_combined
    #return statement was for testing purposes it works awesomesauce
    
    

In [ ]:
parent_sku_list = []
for sku in warranty_DC['parent_Parent_Sku']:
    if sku not in parent_sku_list:
        parent_sku_list.append(sku)


for sku in parent_sku_list:
    sku_df = failure_model_w_CI(sku)